In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
class Head(nn.Module):
    def __init__(self,n_embd,head_size,dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        B, T, C = x.shape # x has shape (batch_size, sequence_length, n_embd)
        q = self.query(x)  # (B, T, head_size)
        k = self.key(x)    # (B, T, head_size)
        v = self.value(x)  # (B, T, head_size)
        out = F.scaled_dot_product_attention(
            q, k, v,
            is_causal=True,  # This enforces the causal mask automatically
            dropout_p=self.dropout.p if self.training else 0.0
        )
        
        return out 

    


In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd,num_heads,dropout):
        super().__init__()
        assert n_embd % num_heads == 0, "n_embd must be divisible by num_heads"

        self.num_heads = num_heads
        self.head_size = n_embd // num_heads
        self.n_embd = n_embd
        self.heads = nn.ModuleList([Head(n_embd, self.head_size, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([head(x) for head in self.heads],dim = -1)
        out = self.proj(out)
        out =  self.dropout(out)
        return out
    
class FeedForward(nn.Module):
    def __init__(self,n_embd,dropout = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout))
    def forward(self,x):
        return self.net(x)   
class Block(nn.Module):
     def __init__(self, n_embd, num_heads, dropout=0.1):
          super().__init__()
          self.sa = MultiHeadAttention(n_embd, num_heads, dropout)
          self.ffwd = FeedForward(n_embd, dropout)
           # Layer normalization
          self.ln1 = nn.LayerNorm(n_embd)
          self.ln2 = nn.LayerNorm(n_embd)
     def forward(self, x):
          x = x + self.sa(self.ln1(x)) 
          x = x + self.ffwd(self.ln2(x)) 
          return x


        
    


         









In [10]:
n_embd = 64
num_heads = 4
block = Block(n_embd, num_heads).to(device)

test_input = torch.randn(2, 10, n_embd).to(device)
output = block(test_input)
print(f"Block input shape: {test_input.shape}")
print(f"Block output shape: {output.shape}")
print(f"Transformer Block successfully created!")

Block input shape: torch.Size([2, 10, 64])
Block output shape: torch.Size([2, 10, 64])
Transformer Block successfully created!


Token Embedding Layer: Converts token indices (integers) into dense vectors, just like the Bigram model. Each token in the vocabulary gets mapped to a learnable embedding vector.

Positional Embedding Layer: Since attention treats all tokens equally, we need to give the model a sense of token order. Positional embeddings encode the position of each token in the sequence, allowing the model to understand "first word", "second word", etc.

Stack of Transformer Blocks: Multiple blocks (typically 6-12 for small models, 96+ for large models) stacked on top of each other. Each block refines the understanding of the sequence.

Final LayerNorm: Normalizes the final representations before the output layer.

Output Linear Layer: Maps the final hidden representations back to vocabulary logits (scores for each token in the vocabulary).

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size,n_embd,block_size,num_heads,num_layers,dropout=0.1):
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, num_heads, dropout) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(n_embd)  # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)  # Language model head
        
        # Better initialization
        self.apply(self._init_weights)

    def _init_weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean = 0.0,std= 0.02)
            if module.bias is None:
                 torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)  

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)  # (B, T, n_embd)
        x = self.ln_f(x)  # (B, T, n_embd)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    

    